In [66]:
import sys
sys.path.append("../python")

import json
import h5py
import numpy as np
import arviz as az
import compute_costs
import evaluations

In [67]:
settings = [
    "MH",
    #"MH_1",
    #"MH_2"#,
    #"MH_2"
    "MLDA ref"
    #"MLDA test"
]

parameters_paths = [
    "../configs/test_mh/mh.json",
    #"../configs/test_mh/mh_1.json",
    #"../configs/test_mh/mh_2.json"#,
    #"../configs/test_mh/mh_2.json"
    "../configs/test_mlda/mlda_ref.json",
    #"../configs/test_mlda/mlda_test.json"
]

samples = []
chains = []
num_samples = []
burn_in = []
costs = []
    
for path in parameters_paths:
    file = open(path)
    config = json.load(file)
    file.close()

    num_samples.append(config["Sampling"]["NumSamples"])
    burn_in.append(config["Sampling"]["BurnIn"])
    chains.append(config["Setup"]["Chains"])
    
    samples_mlda = []
    for c in range(config["Setup"]["Chains"]):
        sample_path = config["Setup"]["OutputPath"] + config["Sampling"]["ResultFile"] 
        fin = h5py.File(sample_path + "_" + str(c) + ".h5")
        samples_mlda.append(np.array(fin['/samples'][:,:]))
        fin.close()
    samples.append(samples_mlda)

In [68]:
# compute ess
ess = []

for i,sample in enumerate(samples):
    ess_mlda = []
    for j,chain in enumerate(sample): 
        if(evaluations.is_converged(chain)==False): 
            print("Not converged in setting " + settings[i] + " for chain " + str(j))  
        elif(evaluations.burn_in_check(chain)==False): 
            print("Burn-in too small in setting " + settings[i] + " for chain " + str(j))  
        ess_mlda.append(min(az.ess(chain[0]),az.ess(chain[1]),az.ess(chain[2])))
    ess.append(ess_mlda)



Burn-in too small in setting MH for chain 8
Burn-in too small in setting MH for chain 9
Burn-in too small in setting MLDA ref for chain 5
Not converged in setting MLDA ref for chain 7
Burn-in too small in setting MLDA ref for chain 8
Not converged in setting MLDA ref for chain 9


In [69]:
print("Samples (on finest level) per effective sample \n")

samples_per_es = np.zeros((len(settings),min(chains)))
rel_samples_per_es = np.zeros((len(settings),min(chains)))

for j in range(min(chains)):
    for i in range(len(settings)):
        samples_per_es[i,j] = (num_samples[i]-burn_in[i])/ess[i][j]
        rel_samples_per_es[i,j] = samples_per_es[i,j]/samples_per_es[0,j]

        print(settings[i] + ": " + str(samples_per_es[i,j]) + " (" + str(rel_samples_per_es[i,j]) +")")
            
            



Samples (on finest level) per effective sample 

MH: 1786.666787374159 (1.0)
MLDA ref: 1021.7995070205068 (0.5719026705154308)
MH: 667.0491813997663 (1.0)
MLDA ref: 723.6125659728905 (1.084796423037997)
MH: 1045.1377963522061 (1.0)
MLDA ref: 648.3451787044853 (0.6203442081679307)
MH: 4243.775767975052 (1.0)
MLDA ref: 4604.439885614862 (1.0849866103580452)
MH: 1511.263464195059 (1.0)
MLDA ref: 1486.6374246295395 (0.9837049990627306)
MH: 1168.6645074855305 (1.0)
MLDA ref: 1591.6075052907665 (1.3619028344714854)
MH: 66351.47424276288 (1.0)
MLDA ref: 277.4589123508625 (0.004181654070499053)
MH: 1487.568321500171 (1.0)
MLDA ref: 25031.03428405286 (16.826813210710057)
MH: 17961.85890779294 (1.0)
MLDA ref: 6603.798424361808 (0.36765673632458395)
MH: 44161.438625762574 (1.0)
MLDA ref: 57433.87769396862 (1.3005436299455895)


In [70]:
print("Average samples per effective sample \n")

for i in range(len(settings)):
    print(settings[i] + ": " + str(np.mean(samples_per_es[i])) + ", " + str(np.var(samples_per_es[i])))


Average samples per effective sample 

MH: 14038.489760260034, 473318885.572135
MLDA ref: 9942.26113819672, 300833676.87642336


In [71]:
costs = np.zeros(len(parameters_paths))
for i in range(len(settings)):
    costs[i] = evaluations.costs_per_sample(parameters_paths[i])

Dipole:
Dipole[position: [120 220] moment: [0.0707372 0.997495]]
volume_conductor.time_creating_grid = 3.06865
volume_conductor.time_reordering_labels = 0.0061425
volume_conductor.time = 3.0748
time_element_neighborhood_map 0.0244613 s
degree = 1
element_type = hexahedron
Dipole[position: [120 220] moment: [0.0707372 0.997495]]
sigma = 1.150841005253733
{'Level3_V': {0: array([ -8.364491  ,  -7.56593475,  -7.69006953,  -7.43026017,
        -9.37340315,  -8.20991605,  -8.33883468, -10.98090956,
        -8.08612719, -10.75063096,  -7.29940048,  -9.56235824,
        -8.97466772,  -9.08452417, -10.51910905,  -7.68911958,
        -9.88051891,  -7.46337842,  -5.02643928,  -4.49160955,
         1.63831439,   5.41082228,  15.98472271,  21.11142897,
        24.54329731,  22.64833662,  19.91082631,  16.64005181,
        14.90886761,  12.02387139,  11.19154262,   4.43935654,
         1.26481454,   2.12115159,   1.72082684,   1.27733858])}}
volume_conductor.time_creating_grid = 0.388906
volume_con

In [72]:
print("Costs per effective sample \n")

costs_per_ed = np.zeros((len(settings),min(chains)))
rel_costs_per_ed = np.zeros((len(settings),min(chains)))

for j in range(min(chains)):
    for i in range(len(settings)):
        costs_per_ed[i,j] = costs[i]/ess[i][j]
        rel_costs_per_ed[i,j] = costs_per_ed[i,j] / costs_per_ed[0,j]

        print(settings[i] + ": " + str(costs_per_ed[i,j]) + " (" + str(rel_costs_per_ed[i,j]) + ")")
            
    print("\n")

Costs per effective sample 

MH: 15.338795986140942 (1.0)
MLDA ref: 31.844337868245944 (2.076065024726729)


MH: 5.726714896430566 (1.0)
MLDA ref: 22.551354623120478 (3.937921658571938)


MH: 8.972661018237313 (1.0)
MLDA ref: 20.205649722925116 (2.2519127471612133)


MH: 36.43343637207604 (1.0)
MLDA ref: 143.49717180731102 (3.938612057941717)


MH: 12.974418130123405 (1.0)
MLDA ref: 46.33099165953304 (3.5709494788027434)


MH: 10.033155920981724 (1.0)
MLDA ref: 49.60237972702215 (4.943846195322424)


MH: 569.6371220316923 (1.0)
MLDA ref: 8.646995118661355 (0.015179830780375774)


MH: 12.770991860475469 (1.0)
MLDA ref: 780.0911112761281 (61.08304819224002)


MH: 154.20518882729252 (1.0)
MLDA ref: 205.8070950262723 (1.3346314517131659)


MH: 379.1324170359701 (1.0)
MLDA ref: 1789.924338193623 (4.721106024610405)




In [73]:
print("Average relative costs per effective sample \n")

for i in range(len(settings)):
    print(settings[i] + ": " + str(np.mean(rel_costs_per_ed[i])) + ", " + str(np.var(rel_costs_per_ed[i])))


Average relative costs per effective sample 

MH: 1.0, 0.0
MLDA ref: 8.787327266187074, 306.0632229582447
